In [1]:
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
import torch
import torch
from diffusers import StableDiffusionPipeline, DDIMScheduler
from attentionControl import AttentionControlEdit
import diff_latent_attack
from PIL import Image
import numpy as np
import os
import glob
from other_attacks import model_transfer
import random
import sys
from natsort import ns, natsorted
import argparse

In [ ]:
repo_id = "stabilityai/stable-diffusion-2-base"
pipe = DiffusionPipeline.from_pretrained(repo_id, torch_dtype=torch.float16, revision="fp16")

pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")

prompt = "Best quality, masterpiece, ultra high res, (photorealistic:1.4), raw photo, 1girl, offshoulder, in the dark, deep shadow, low key, cold light"
image = pipe(prompt, num_inference_steps=50).images[0]
image.save("astronaut.png")

In [ ]:
pretrained_diffusion_path = "stabilityai/stable-diffusion-2-base"
ldm_stable = StableDiffusionPipeline.from_pretrained(pretrained_diffusion_path).to('cuda:0')
ldm_stable.scheduler = DDIMScheduler.from_config(ldm_stable.scheduler.config)

In [ ]:
code_all = ldm_stable.tokenizer("hello world xxxxxxxxfdgdfgdgxxxxxxxxxasfsdfsfdf")
code = ldm_stable.tokenizer.encode("hello world xxxxxxxxfdgdfgdgxxxxxxxxxasfsdfsfdf")
code_str = ldm_stable.tokenizer.decode(code)

print(code_all)
print(code)
print(code_str)


In [6]:
! export https_proxy=http://turbo.gpushare.com:30000 http_proxy=http://turbo.gpushare.com:30000

from other_attacks import model_selection

# models_transfer_name = ["resnet", "vgg", "mobile", "inception", "convnext", "vit", "swin", 'deit-b', 'deit-s',
#                             'mixer-b', 'mixer-l', 'tf2torch_adv_inception_v3', 'tf2torch_ens3_adv_inc_v3',
#                             'tf2torch_ens4_adv_inc_v3', 'tf2torch_ens_adv_inc_res_v2']
models_transfer_name = ["resnet", "vgg", "mobile", "inception", "convnext", "vit", "swin", 'deit-b', 'deit-s',
                            'mixer-b', 'mixer-l']

for name in models_transfer_name:
    model = model_selection(name)

In [20]:
start_value = 0.0001
end_value = 0.01
num_elements = 40

b = torch.linspace(start_value, end_value, num_elements)
a = torch.ones_like(b) - b
a_bar = torch.cumprod(a,0)

n = torch.randn(100)
b = n*torch.sqrt(1-a_bar[20])

b

tensor([-0.1123, -0.0604, -0.0974, -0.0652, -0.0022, -0.3359,  0.1098,  0.0798,
         0.3170,  0.4750,  0.0655, -0.0515,  0.2791,  0.0928,  0.0677,  0.1148,
        -0.1695, -0.0696,  0.2180,  0.0454,  0.5167, -0.0930, -0.2801, -0.3299,
        -0.3137, -0.0812,  0.0168, -0.2425,  0.0949, -0.0797,  0.1307, -0.0897,
        -0.0370,  0.0398, -0.0651, -0.0665, -0.0793,  0.0869,  0.0932,  0.2013,
        -0.0750,  0.0680,  0.0773,  0.1288,  0.2136, -0.2136, -0.1233, -0.2757,
        -0.1075,  0.1196, -0.2139, -0.3380,  0.2157, -0.2172, -0.1967,  0.0083,
        -0.0950,  0.2830,  0.2774, -0.1848,  0.2398,  0.1096,  0.0351, -0.0164,
         0.1670,  0.0598, -0.0179, -0.2654,  0.1957,  0.1590,  0.0766, -0.2397,
        -0.0390, -0.1565, -0.2180, -0.0020,  0.0602, -0.1038,  0.0813, -0.0424,
         0.3504, -0.0687, -0.0176, -0.0359, -0.1863,  0.0670, -0.0657,  0.0969,
        -0.2065,  0.1179,  0.1353,  0.1265, -0.0668,  0.3650,  0.1122,  0.1341,
         0.0256,  0.1060, -0.1015, -0.07